In [1]:
import os
%pwd

'/workspaces/BrainTumorSegmentation/research'

In [2]:
os.chdir("../")

In [ ]:

%pwd

'/workspaces/BrainTumorSegmentation'

In [ ]:
# entity > confit_entity.py
# entity is return type of any function... here is return type of data ingestion function

from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [5]:
# importing yaml file  path located on constant inti file
from BrainTumorReport.constants import *
# importing read_yaml and create_directories from utils common code
from src.BrainTumorReport.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
# components
import os
import urllib.request as request
import zipfile
import gdown
from src.BrainTumorReport import logger
from src.BrainTumorReport.utils.common import get_size

In [7]:
# main logic code
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)->str:

        try:
            dataset_url = self.config.source_URL
            zip_doownload_dir=self.config.local_data_file
            os.makedirs("artifacts/data_ingestion",exist_ok=True)
            logger.info(f"Downloading file from {dataset_url} into file  {zip_doownload_dir}")


            file_id=dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_doownload_dir)
            logger.info(f"File Downloaded at {zip_doownload_dir}")

        except Exception as e:
            raise e
    
    def extract_zip_file(self):
        unzip_dir=self.config.unzip_dir
        os.makedirs(unzip_dir,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_dir)
        logger.info(f"Unzipped file at {unzip_dir}")

    def delete_zip_file(self):
        os.remove(self.config.local_data_file)
        logger.info(f"Deleted zip file at {self.config.local_data_file}")
    

In [8]:
# pipeline

try:
    config=ConfigurationManager()
    DataIngestionConfig=config.get_data_ingestion_config()
    dataIngestion=DataIngestion(config=DataIngestionConfig)
    dataIngestion.download_file()
    dataIngestion.extract_zip_file()
    dataIngestion.delete_zip_file()
except Exception as e:
    raise e


[2025-07-19 18:40:00,910: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-07-19 18:40:00,912: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-19 18:40:00,913: INFO: common: created directory at: artifacts]
[2025-07-19 18:40:00,914: INFO: common: created directory at: artifacts/data_ingestion]
[2025-07-19 18:40:00,915: INFO: 2192072649: Downloading file from https://drive.google.com/file/d/15u8adG6K8tpWAX6ilvyTXKyjqhZcUiPe/view?usp=sharing into file  artifacts/data_ingestion/BTS.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=15u8adG6K8tpWAX6ilvyTXKyjqhZcUiPe
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=15u8adG6K8tpWAX6ilvyTXKyjqhZcUiPe&confirm=t&uuid=b6fc4603-b762-4032-a5ff-1f48178b3758
To: /workspaces/BrainTumorSegmentation/artifacts/data_ingestion/BTS.zip
100%|██████████| 47.9M/47.9M [00:01<00:00, 39.6MB/s]

[2025-07-19 18:40:04,668: INFO: 2192072649: File Downloaded at artifacts/data_ingestion/BTS.zip]


[2025-07-19 18:40:04,896: INFO: 2192072649: Unzipped file at artifacts/data_ingestion]
[2025-07-19 18:40:04,908: INFO: 2192072649: Deleted zip file at artifacts/data_ingestion/BTS.zip]
